In [1]:
# Useful starting lines
import numpy as np
from dataloader import *
from finance import *
from plots import *
from finance import stock, compare
from quotebankexploration import *
from wikipedia import *
%load_ext autoreload
%autoreload 2

***
## Quotebank Data Exploration

In [ ]:
quotes = load_quotes(2020, 'unprocessed quotes')
quotebank_exploration(quotes)

The dataset comprises the following columns : 
 * `quoteID`: Primary key of the quotation (format: "YYYY-MM-DD-{increasing int:06d}").
 * `quotation`: Text of the longest encountered original form of the quotation.
 * `speaker`: Selected most likely speaker. This matches the first speaker entry in `probas`. If none of the speakers is selected, the speaker is defined as "None".
 * `qids`: Wikidata IDs of all aliases that match the selected speaker. If no Wikidata IDs is found, the value is '[]'.
 * `date`: Earliest occurrence date of any version of the quotation.
 * `numOccurrences`: Number of time this quotation occurs in the articles.
 * `probas`: Array representing the probabilities of each speaker having uttered the quotation.
 * `urls`: List of links to the original articles containing the quotation. 
 * `phase`: Corresponding phase of the data in which the quotation first occurred (A-E).

***
## Quotebank Data Pre-processing
We filter and transform the data according to our needs. The first step is to filter out any quotes that are not related to the company, the products or its direction board. To do so, we use a combination of white-listed words (Apple, iPhone, iPad, Macbook etc.), black-listed words that are not related to the Apple company (Big Apple, apple pie, big mac etc.) and relevant speakers (Tim Cook, Steve Jobs etc.). The filtered dataset is then saved as a pickle file.

In [ ]:
# Key-words for filtering

# One word key-words
keywords_1_word = ["iphone", "ipad", "imac", "ipod", "macbook", "mac", "airpods",
        "lightening", "magsafe", "aapl", "iwatch", "itunes", "homepod", "macos", "ios", "ipados",
        "watchos", "tvos", "wwdc", "siri", "facetime", "appstore", "icloud", "iphones"]

# Two words key-words
keywords_2_words = ["apple watch", "steve jobs", "tim cook", "face id",
        "pro display xdr", "katherine adams", "eddy cue", "craig federighi"]

# Keep only the 'Apple' that corresponds to the company
keywords_capital = ["Apple"]

# Remove all the words that can be confused with the words we are looking for
black_list = ["Big Apple"]

# Create a dictionnary of key-words
keywords = {"One word": keywords_1_word, "Two words": keywords_2_words, "Capital words": keywords_capital, "Black list": black_list}

# Speakers we want to look for quotes
speakers = ["steve jobs", "tim cook", "katherine adams", "eddy cue", "craig federighi", "john giannandrea", "greg joswiak",
    "sabih khan", "luca maestri", "deirdre o'brien", "johny srouji", "john ternus", "jeff williams", "lisa jakson",
    "stella low", "isabel ge mahe", "tor myhren", "adrian perica", "phil schiller", "arthur levinson", "james bell",
    "albert gore", "andrea jung", "monica lozano", "ronald sugar", "susan wagner"]

In [ ]:
# The idea of this section is to now apply the filter with the key-words
# we define before.

# Initialize the lists for the final plots
frequency_all = []
frequency_apple = []
years = np.arange(2008, 2009)

# Loop over all the years from 2008 to 2020
for year in years:
    # Filtering the quotes
    filt_quotes = filter_quotes(f"data/quotes-{str(year)}.json.bz2", \
        speakers = speakers, keywords = keywords, save = f"filtered_quotes_{str(year)}")
    
    # Update the list for the final plots
    frequency_all.append(filt_quotes['total'])
    frequency_apple.append(len(filt_quotes['dataframe']))
    

#### Distribution of the unprocessed and processed quotes among the years

Now we have filtered all the quotes we wants to keep for our following study, we want to know the distribution of the quotes among the years and how many quotes we deal with. The idea is also to see if there is a particular year where we have significantly more or less quotes than the others. Then, in the following figure we have plotted the total number of quotes and the number of quotes we have after filtering for every year from `2008` to `2020`.

Now, when we look the graph we have that `[... interpretation ...]` --> J'attends que Baptiste update tous les keywords qu'il y a à mettre en blacklist pour avoir les bon chiffres...

In [ ]:
# Plot the results
bar_plots_quotes(frequency_all, frequency_apple, years)

### EDA after filtering

This new filtered dataset provides a great preview of Apple mentions in the media. Let's look more precisely in this dataset :

In [ ]:
year = 2019
quote_df = load_quotes(year,category= 'processed quotes')
quotebank_exploration(quote_df)

We noticed that they were some quotes that were still irrelevant due to the keyword "mac" so we decided to filter again a little our dataset by removing couple of words as "big mac", "freddie mac".

In [ ]:
quote_df = refilter(quote_df)

In [ ]:
print("Total number of occurences from every quotes of each speaker for year %i" %year)
df_table = plot_table_numOcc(quote_df)
df_table

In [ ]:
plot_pie_numquote(quote_df, year, head_ = 5)

In [ ]:
plot_quotes_per_day(quote_df, year)

In [ ]:
plot_numOcc_per_day(quote_df, year)

***

# Wikipedia

For now, we have our data set with some quotes more or less connected to Apple and the speaker corresponding to each quote. Now, we want to know what is the "impact" of the speaker. We want to know how much this person is famous or/and has an impact on the others. For that, we will use Wikipedia that has almost a page for every famous person. So the idea now is to get the data set coming from the Wikipedia API, study it and see how we can use it for our problem.

For now, we will use the `.parquet` already given from the Wikipedia API and we will see later if we need more information than the already given ones. The first idea will be to group together all `.parquet` files in one single dataframe.

In [2]:
# Get all the parquet files in a single dataframe
df_wiki = concat_wiki_files()
display(df_wiki.head(10))

,aliases,date_of_birth,nationality,gender,lastrevid,ethnic_group,US_congress_bio_ID,occupation,party,academic_degree,id,label,candidacy,type,religion
0,"[Washington, President Washington, G. Washingt...",[+1732-02-22T00:00:00Z],"[Q161885, Q30]",[Q6581097],1395141751,None,W000178,"[Q82955, Q189290, Q131512, Q1734662, Q294126, ...",[Q327591],None,Q23,George Washington,"[Q698073, Q697949]",item,[Q682443]
1,"[Douglas Noel Adams, Douglas Noël Adams, Dougl...",[+1952-03-11T00:00:00Z],[Q145],[Q6581097],1395737157,[Q7994501],None,"[Q214917, Q28389, Q6625963, Q4853732, Q1884422...",None,None,Q42,Douglas Adams,None,item,None
2,"[Paul Marie Ghislain Otlet, Paul Marie Otlet]",[+1868-08-23T00:00:00Z],[Q31],[Q6581097],1380367296,None,None,"[Q36180, Q40348, Q182436, Q1265807, Q205375, Q...",None,None,Q1868,Paul Otlet,None,item,None
3,"[George Walker Bush, Bush Jr., Dubya, GWB, Bus...",[+1946-07-06T00:00:00Z],[Q30],[Q6581097],1395142029,None,None,"[Q82955, Q15982858, Q18814623, Q1028181, Q1408...",[Q29468],None,Q207,George W. Bush,"[Q327959, Q464075, Q3586276, Q4450587]",item,"[Q329646, Q682443, Q33203]"
4,"[Velázquez, Diego Rodríguez de Silva y Velázqu...",[+1599-06-06T00:00:00Z],[Q29],[Q6581097],1391704596,None,None,[Q1028181],None,None,Q297,Diego Velázquez,None,item,None
5,"[Augusto Pinochet Ugarte, Augusto José Ramón P...",[+1915-11-25T00:00:00Z],[Q298],[Q6581097],1392242213,None,None,"[Q189290, Q82955]",[Q327591],None,Q368,Augusto Pinochet,None,item,[Q1841]
6,"[Baudelaire, Charles Pierre Baudelaire-Dufaÿs,...",[+1821-04-09T00:00:00Z],[Q142],[Q6581097],1386699038,[Q121842],None,"[Q49757, Q4164507, Q11774202, Q333634, Q36180,...",None,None,Q501,Charles Baudelaire,None,item,[Q1841]
7,"[Mikołaj Kopernik, Nikolaus Kopernikus, Copern...",[+1473-02-19T00:00:00Z],[Q1649871],[Q6581097],1394975677,[Q1026],None,"[Q11063, Q185351, Q188094, Q170790, Q16012028,...",None,None,Q619,Nicolaus Copernicus,None,item,[Q1841]
8,"[Neil Percival Young, Shakey, Godfather of Gru...",[+1945-11-12T00:00:00Z],"[Q16, Q30]",[Q6581097],1395459626,None,None,"[Q177220, Q488205, Q2526255, Q639669, Q1881462...",None,None,Q633,Neil Young,None,item,None
9,None,[+1969-00-00T00:00:00Z],[Q183],[Q6581097],1340253739,None,None,"[Q33231, Q41546637]",None,None,Q640,Harald Krichel,None,item,None


This being done, we can see multiple features for each speaker. An interesting one for us is the `id` that will corresponds to the `qids` of the QuoteBank data set. The idea will be to keep only the speakers in the Wikipedia dataframe that are in our filtered quotes data set. Then, we will get all the speakers' ID that have spoken in our filtered quotes data set.

In [3]:
''' !!! MAYBE CHANGE THE PLACE OF THIS CELL !!! '''
# Here we group together all the filtered quotes in a single dataframe
df_filtered = get_filtered_quotes()
display(df_filtered.head(10))

,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase
0,2008-10-10-007764,congress pushed fannie mae and freddie mac to ...,george w. bush,[Q207],2008-10-10 05:19:22,1,"[[george w. bush, 0.6274], [None, 0.3726]]",[http://neworleanscitybusiness.wordpress.com/2...,A
1,2008-10-15-012147,five times the graphics performance.,steve jobs,[Q19837],2008-10-15 06:38:46,2,"[[steve jobs, 0.7707], [None, 0.2293]]",[http://us.rd.yahoo.com/dailynews/rss/search/m...,A
2,2008-09-25-038377,my iphone is full of pictures. i wanted to see...,ken whisenhunt,[Q1758635],2008-09-25 08:08:08,1,"[[ken whisenhunt, 0.3759], [None, 0.3186], [ma...",[http://azcentral.com/arizonarepublic/sports/a...,A
3,2008-09-04-052789,the people who connect needs and ideas the bes...,steve jobs,[Q19837],2008-09-04 18:19:47,1,"[[steve jobs, 0.717], [None, 0.283]]",[http://businessweek.com/magazine/content/07_4...,A
4,2008-11-10-038937,sesame workshop will get 70 percent of the rev...,robert macmillan,"[Q21453558, Q28094302]",2008-11-10 22:59:44,1,"[[robert macmillan, 0.6071], [None, 0.3929]]",[http://macdailynews.com/index.php/weblog/comm...,A
5,2008-09-26-048683,the huge success of the apple itunes app store...,steve howard,"[Q23670647, Q2457386, Q7612886]",2008-09-26 11:29:23,1,"[[steve howard, 0.5034], [None, 0.4966]]",[http://ipod.consumerelectronicsnet.com/articl...,A
6,2008-11-24-055069,we have a tough game against a tough opponent....,doug martin,"[Q18685889, Q27995830, Q3037945, Q3714598, Q53...",2008-11-24 21:48:30,1,"[[doug martin, 0.5937], [None, 0.4063]]",[http://kentstatesports.com//ViewArticle.dbml?...,A
7,2008-12-19-024329,it just takes the right kind of auto battery -...,steve jobs,[Q19837],2008-12-19 15:46:22,1,"[[steve jobs, 0.2931], [thomas friedman, 0.287...",[http://jpost.com/servlet/Satellite?cid=122872...,A
8,2008-09-26-058206,"until the current credit crisis, mr. schumer h...",chuck schumer,[Q380900],2008-09-26 21:43:57,1,"[[chuck schumer, 0.4751], [barack obama, 0.198...",[http://weblogs.newsday.com/news/local/longisl...,A
9,2008-10-08-017621,i like so far that [ obama ] has highlighted t...,barack obama,[Q76],2008-10-08 03:27:01,1,"[[barack obama, 0.3835], [john mccain, 0.2181]...",[http://keyetv.com/news/local/story.aspx?conte...,A


In [4]:
# Now we keep only the 'qids' and we remove duplicates
quotes_ID = get_wiki_ids(df_filtered)

In [5]:
pd.DataFrame(quotes_ID)

,qids
0,"b""['Q207']"""
1,"b""['Q19837']"""
2,"b""['Q1758635']"""
4,"b""['Q21453558', 'Q28094302']"""
5,"b""['Q23670647', 'Q2457386', 'Q7612886']"""
...,...
4550,"b""['Q4717192']"""
4554,"b""['Q13486477']"""
4556,"b""['Q26832781', 'Q56098687', 'Q57733391']"""
4558,"b""['Q16210933']"""


Now, we have to find a way to get a correspondance between the `qids` of the quotes and the `id` of the Wikipedia data set. This will informs us if there is some speaker that are not in our Wikipedia data set, and then if we must find another way to obtain the informations we want.

[in progress...]

***

# Yahoo Finance Data Exploration

The yfinance library provides financial metrics on most stocks and indices. It will allow us to quantitatively compare the stock price and volume of Apple to a major equity index, such that the S&P500. Later on we will use this new dataset to find and compare days of high volatility, and find any correlation with potential speakers and quotations.

A stock dataframe consists of the following features: 
 * `Date`: Primary key of the quotation (format: "YYYY-MM-DD").
 * `Open`: Stock price at the daily opening of the market.
 * `High`: Highest stock price reached during the day.
 * `Low`: Lowest stock price reached during the day.
 * `Adj Close`:  Adjusted stock price at the daily close of the market.
 * `Volume`: Amount of stocks that changed hands during the day.

 We will focus our study on the `Adjusted Open`, `Adjusted Close` and `Volume`.

In [ ]:
apple_ticker = "AAPL"
year = 2019
apple_stock = yf.download(apple_ticker, start=f'{year}-01-01', end=f'{year}-12-31', progress = False)
apple_stock.head()

In [ ]:
''' !!! LEGEND !!! '''

stock("AAPL", year = year, fig = "price_volume")

In [ ]:
stock("AAPL", year = year, fig = "daily_diff")

In [ ]:
stock("AAPL", year = year, fig = "volume")

In [ ]:
compare("^GSPC", "AAPL", year = year)

***
### Subtask 1 : 
What is the role of the media coverage in explaining stock market fluctuations ? How does the media coverage of Apple evolve over time ? Are there any specific patterns we can observe across the years, for instance bigger coverage when there is the keynote ? Thousands of events occur around the world every day, and humans notice a small subset of these events. Yet, the media attract attention to specific events. In this regard, a company’s media visibility will definitely affect its stock price. We investigate the reciprocal relationships between the fluctuations of the stock market of Apple and media coverage related to this company for a period of 12 years (2008–2020). 

***
### Subtask 2 
Who are the individuals who have influence over potential customers, and do these influencers have an impact on the Apple company image and eventually, on the stock market ? Personality  plays  a huge  role  in  consumer  buying  behavior. Indeed, the high level of public attention and the positive emotional responses that define celebrity increase the economic opportunities available to a firm. We hypothesize that quotes from celebrities significantly impact the stock market, whereas quotes from ordinary people have no significant predictive power. One defining characteristic of a celebrity is that it is a social actor who attracts large-scale public attention : the greater the number of people who know of and pay attention to the actor, the greater the extent and value of that celebrity.

***

### Subtask 3 :
What is the influence of the public opinions or emotions about Apple expressed in the media on the stock market ? Media influences beliefs by providing compelling information about events. In this regard, the media have been identified to play a significant role in shaping the consensus market opinion. Can a series of news articles make a stock rise? On the other hand, can it send a market into turmoil ? We hypothesize that emotions of the public in the media about a company would reflect in its stock price. Indeed, positive news about Apple would encourage people to invest in the stocks of the company. On the other hand, increases in expressions of anxiety, worry and fear would predict downward pressure on the stocks of the company. Understanding the author's opinion from a piece of text is the objective of sentiment analysis. We classify quotes as positive, negative and neutral based on the sentiment present. We want to answer the following questions : are the stock price drops/rises correlated with negative/positive quotes ? 